In [1]:
%pip uninstall qiskit qiskit-terra qiskit-aer qiskit-machine-learning -y

%pip install qiskit==0.44.2
%pip install qiskit-aer==0.12.
%pip install qiskit-machine-learning==0.6.1

%pip install torch torchvision matplotlib


Found existing installation: qiskit 0.44.2
Uninstalling qiskit-0.44.2:
  Successfully uninstalled qiskit-0.44.2
Found existing installation: qiskit-terra 0.25.2.1
Uninstalling qiskit-terra-0.25.2.1:
  Successfully uninstalled qiskit-terra-0.25.2.1
Found existing installation: qiskit-aer 0.12.2
Uninstalling qiskit-aer-0.12.2:
  Successfully uninstalled qiskit-aer-0.12.2
Note: you may need to restart the kernel to use updated packages.


  Using cached qiskit-0.44.2-py3-none-any.whl (8.2 kB)
  Using cached qiskit_terra-0.25.2.1-cp38-abi3-win_amd64.whl (5.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached qiskit_aer-0.12.0-cp310-cp310-win_amd64.whl (9.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached qiskit_machine_learning-0.6.1-py3-none-any.whl (148 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# ============================================================
# IMPORTS
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
import numpy as np

# Qiskit (VERSION COMPATIBLE)
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.circuit import ParameterVector # Necesario para definir inputs y pesos
from qiskit.opflow import Z, I 

# ============================================================
# 1. PREPARAR MNIST REDUCIDO A 4 CARACTERÍSTICAS
# ============================================================

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_data = datasets.MNIST(root="./data", train=True, transform=transform, download=True)

# Tomamos SOLO 1000 imágenes para no durar horas
subset_indices = list(range(1000))
mnist_small = Subset(mnist_data, subset_indices)

train_loader = DataLoader(mnist_small, batch_size=32, shuffle=True)


# Función para reducir 28x28 -> 4 valores
def reduce_to_4_features(img):
    img = img.view(28, 28)
    return torch.tensor([
        img[:14, :14].mean(),   # cuadrante 1
        img[:14, 14:].mean(),   # cuadrante 2
        img[14:, :14].mean(),   # cuadrante 3
        img[14:, 14:].mean()    # cuadrante 4
    ])


# ============================================================
# 2. CONSTRUIR EL QNN CUÁNTICO (VERSIÓN CORREGIDA FINAL)
# ============================================================


def create_qnn(n_qubits=4):
    qc = QuantumCircuit(n_qubits)
    input_params = ParameterVector('x', n_qubits)
    weight_params = ParameterVector('w', n_qubits)
    
    # [Circuit definition remains the same]
    for i in range(n_qubits):
        qc.rx(input_params[i], i)
        qc.ry(weight_params[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
        
    # --- Definición de Observables ---
    # Queremos 4 salidas, así que medimos Z en cada qubit por separado.
    # El observable será: [Z_0, Z_1, Z_2, Z_3]
    observables = [Z ^ (I * (n_qubits - 1 - i)) for i in range(n_qubits)]

    estimator = Estimator()

    qnn = EstimatorQNN(
        estimator=estimator,
        circuit=qc,
        input_params=list(input_params),
        weight_params=list(weight_params),
        # *** CORRECCIÓN CLAVE ***:
        # Pasa la lista de 4 observables. Esto garantiza 4 salidas.
        observables=observables 
    )

    return qnn

# ============================================================
# 3. MODELO HÍBRIDO PyTorch + QNN
# ============================================================

class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(4, 4)

        # La instancia QNN ahora está correctamente configurada
        qnn = create_qnn() 
        self.quantum = TorchConnector(qnn)

        # La salida del QNN por defecto será de tamaño 4 (una expectativa por qubit)
        self.fc_out = nn.Linear(4, 10) 

    def forward(self, x):
        x = x.view(-1, 4)
        x = torch.tanh(self.fc1(x))
        x = self.quantum(x)
        x = self.fc_out(x)
        return F.log_softmax(x, dim=1)

# Inicialización del modelo (debe ejecutarse después de las definiciones)
model = HybridModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# ============================================================
# 4. ENTRENAMIENTO
# ============================================================

losses = []

print("\nEntrenando modelo híbrido cuántico...\n")

for epoch in range(5):  # 5 epochs para mostrar rápido
    total_loss = 0

    for images, labels in train_loader:
        feats = torch.stack([reduce_to_4_features(img) for img in images])

        optimizer.zero_grad()
        output = model(feats) # ¡Esto ya no debería fallar!
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    losses.append(total_loss)
    print(f"Epoch {epoch+1}, loss = {total_loss:.4f}")

print("\nEntrenamiento finalizado ✔\n")


# ============================================================
# 5. GRAFICAR CURVA DE PÉRDIDA
# ============================================================

plt.plot(losses)
plt.title("Curva de pérdida - Modelo Cuántico Híbrido")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


# ============================================================
# 6. PREDICCIÓN DE UNA IMAGEN
# ============================================================

def predict(img):
    x = reduce_to_4_features(img).unsqueeze(0)
    with torch.no_grad():
        pred = model(x)
    return torch.argmax(pred, dim=1).item()


# Tomamos un ejemplo de MNIST
sample_img, sample_label = mnist_data[0]
pred_value = predict(sample_img)

print(f"Etiqueta real: {sample_label}")
print(f"Predicción cuántica: {pred_value}")


Entrenando modelo híbrido cuántico...



RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1 and 4x10)